<a href="https://colab.research.google.com/github/RYOTA2528/data_analysis_practice/blob/main/DAYxxx_python_larning_list/%5BDAY22%5D%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B3%E3%83%B3%E3%83%9A%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E6%A7%8B%E7%AF%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import glob #ファイルのパターン検索
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
files = glob.glob("/content/drive/MyDrive/データコンペ用/data/train/train/*.csv") #ファイルの一括読み込み
# csvファイルの読み込み
data_list = []
for file in files:
  data_list.append(pd.read_csv(file, index_col=0)) #index_col=0で0列目をインデックス
df = pd.concat(data_list)

<ipython-input-24-a1d04737cdb3>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0)) #index_col=0で0列目をインデックス
<ipython-input-24-a1d04737cdb3>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0)) #index_col=0で0列目をインデックス


In [25]:
# データクレンジングを施した処理の関数化
def data_pre(df):
  # nullを除外する
  null_list = []
  for col in df.columns:
    nonnull = df[col].count() #count()でnullは0と表示されるため
    if nonnull == 0:
      null_list.append(col)
  df = df.drop(null_list, axis = 1) # axsis=1で列から除外

  # 上記の結果、市区町村・市区町村コードが同じデータであることがわかる
  # 市区町村コードだけ残します
  df = df.drop("市区町村名", axis=1)
  # 種類も削除します
  df = df.drop("種類", axis=1)

  # "最寄駅：距離（分）" の中の文字列を一括でfloat型にかえる
  dis = {
      "30分?60分": 45,
      "1H?1H30": 75,
      "2H?": 120,
      "1H30?2H": 105
  }

  df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)
  df["最寄駅：距離（分）"].value_counts()

  # 面積を変換
  df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)

  # 建築年数を西暦に変換
  y_list = {}
  for i in df["建築年"].value_counts().keys(): #keyを一つずつ取り出す
    if "平成" in i: #iの中に平成があれば
      num = float(i.split("平成")[1].split("年")[0]) #数字を取り出し格納
      year = 1988 + num #平成なので1988年に足す
    elif "令和" in i:
      num = float(i.split("令和")[1].split("年")[0])
      year = 2018 + num
    elif "昭和" in i:
      num = float(i.split("昭和")[1].split("年")[0])
      year = 1925 + num

    y_list[i] = year

  y_list["戦前"] = 1945

  y_list

  df["建築年"] = df["建築年"].replace(y_list)

  # 取引時点の〇四半期をfloat型に直す処理
  year = {
      "年第１四半期": ".25",
      "年第２四半期": ".50",
      "年第３四半期": ".75",
      "年第４四半期": ".99"
  }

  year_list = {}
  for i in df["取引時点"].value_counts().keys():
    for k, j in year.items(): #year辞書の一つ一つをとりだす
      if k in i: #iにkが入っていたら
        year_rep = i.replace(k, j)
    year_list[i] = year_rep

    year_list[i]

  df["取引時点"] = df["取引時点"].replace(year_list).astype(float)

  # 特徴量（新たなカラム）を追加する
  df["築年数"] = df["取引時点"] - df["建築年"]

  # lightbgmではobject型の読み込みができないため、cate
  # カテゴリカルデータの変換
  for col in ["都道府県名", "地区名", "最寄駅：名称", "最寄駅：距離（分）",
              "間取り", "面積（㎡）", "建築年", "建物の構造", "用途", "今後の利用目的",
              "都市計画", "取引時点", "改装", "取引の事情等"]:
      df[col] = df[col].astype('category')

  return df

In [26]:
df = data_pre(df)
df.head()

<ipython-input-25-9ea5d6a68c08>:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["建築年"] = df["建築年"].replace(y_list)


,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,築年数
ID,,,,,,,,,,,,,,,,,,,
23167208,23222,愛知県,加木屋町,八幡新田,4.0,３ＬＤＫ,75.0,1997.0,ＳＲＣ,住宅,NaN,第１種住居地域,60.0,200.0,2012.75,未改装,NaN,6.991226,15.75
23071718,23226,愛知県,井田町,三郷(愛知),13.0,３ＬＤＫ,50.0,1975.0,ＲＣ,住宅,住宅,第２種中高層住居専用地域,60.0,200.0,2015.75,未改装,NaN,6.342423,40.75
23197007,23116,愛知県,中平,原(愛知),22.0,４ＬＤＫ,100.0,2003.0,ＲＣ,住宅,NaN,第２種住居地域,60.0,200.0,2008.50,未改装,NaN,7.491362,5.50
23132701,23211,愛知県,日之出町,豊田市,11.0,４ＬＤＫ,90.0,2003.0,ＳＲＣ,住宅,NaN,準工業地域,60.0,200.0,2010.25,未改装,NaN,7.397940,7.25
23002332,23102,愛知県,泉,高岳,5.0,１Ｋ,30.0,1986.0,ＳＲＣ,住宅,住宅,商業地域,80.0,500.0,2015.75,未改装,NaN,6.778151,29.75


In [27]:
# lightgbm による機械学習
import lightgbm as lgb
from sklearn.model_selection import train_test_split #訓練データと学習(検証)データに分ける
from sklearn.metrics import mean_absolute_error #評価指標のMAEを簡単に実装できる

df_train, df_val = train_test_split(df, test_size=0.2) #検証データが20％、残りが訓練データ

col = "取引価格（総額）_log"
# 訓練データの設定
# 目的変数
train_y = df_train[col]
# 説明変数
train_x = df_train.drop(col, axis=1) #目的変数colを除外する
# 学習データの設定
# 目的変数
val_y = df_val[col]
# 説明変数
val_x = df_val.drop(col, axis=1) #目的変数colを除外する

# Datasetに変更する
trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

#パラメータ設定
params = {
    "objective": "regression",
    "metrics": "mae"
}

# モデル構築(1000回の学習のうち100ごとにvalid_setsに設定した学習データをもとにmaeで評価そして改善がない場合は打ち切り)
model = lgb.train(
    params,
    trains,
    valid_sets=valids,
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=100)])



[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.088155 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11896
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 18
[LightGBM] [Info] Start training from score 7.217556
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[997]	valid_0's l1: 0.0769956
